##  DSPy 中的评估evaluation

- 学了基本的使用后，我们到了优化前的评估阶段，将从收集初始开发集（development set）到定义指标（metrics）再到评估（evaluation）的整个流程展开。

在 DSPy 中，评估不仅仅是衡量模型好坏的手段，更是迭代和优化模型的核心驱动力。通过评估，我们可以量化模型的表现，发现问题，并指导我们进行改进。这就像是给大模型做“体检”，让我们了解它的“健康状况”，并找到“治疗方案”。

### 评估总流程和的两点核心

1. **收集开发集（Development Set）和定义评估指标函数（Metric）。**

*   **开发集（Development Set）:**  想象一下，开发集就像是我们在构建模型过程中的“模拟考试”。我们需要一些真实的、有代表性的数据来测试我们的模型，以便及时发现问题并进行调整。这个数据集不需要很大，几十个到几百个样本通常就足够了。重要的是，这些样本要能够反映真实应用场景中的数据分布。
    *   **如何获取开发集？**  我们可以从现有的数据集中获取，例如 Hugging Face Datasets 上公开的数据集，或者从一些数据源（如 StackExchange）中收集。如果找不到合适的数据集，我们也可以手动标注一些数据，或者部署一个演示系统来收集用户数据。重要的是，要确保数据的许可允许我们使用它们。 
    
*   **评估指标（Metric）:**  我们需要一把“尺子”来衡量模型的表现。这把“尺子”就是评估指标。评价指标Metric是一个函数，它接收数据样本和 DSPy 程序的输出，并返回一个分数来评估输出的质量。这个分数可以是准确率、F1 分数，或者任何其他能够反映模型性能的指标。
    *   **如何定义一个好的指标？**  定义指标的关键在于理解任务目标。我们需要思考：对于这个任务，什么样的输出是好的？什么样的输出是不好的？然后，我们需要将这些判断标准转化为一个可量化的函数。通常，我们需要从一个简单的指标开始，然后根据实际情况进行迭代改进。
    *   **指标与任务目标有什么关系？**  指标是任务目标的具体体现。一个好的指标应该能够准确地反映模型在实现任务目标方面的表现。简单任务（如分类或简短回答问题）可以用准确率（accuracy）、精确匹配（exact match）或F1分数来表示；而对于更复杂的任务（如需要长文本输出），你可能需要一个更复杂的评估程序，它可能会检查输出的多个维度（例如内容准确性、连贯性、完整性），甚至利用语言模型辅助检查输出质量。例如，如果我们的目标是生成准确的摘要，那么指标就应该评估摘要的准确性。如果我们的目标是生成引人入胜的推文，那么指标就应该评估推文的吸引力。

 2. 运行开发评估 (Run development evaluations)

*   评估的流程具体是什么样子的呢？

    *    我们可以使用Development Set中的数据和定义的Metric指标函数来运行评估。评估的过程通常包括：使用Development Set中的数据运行 DSPy 程序，获取输出，然后使用Metric指标函数计算输出的分数。
    *   **评估的目的是什么？**  评估的目的是了解不同设计的优缺点，发现模型存在的问题，并为后续的改进提供基线。通过查看输出和指标分数，我们可以找到模型需要改进的地方。


##  DSPy 中的数据（DATA）

在 DSPy 中，数据扮演着至关重要的角色。它不仅仅是训练模型的“燃料”，更是评估和优化模型的“指南针”。DSPy 强调数据的有效利用，即使是少量的数据，也可以通过合理的处理和利用，发挥巨大的作用。


### 1. DSPy `Example` 对象

`Example` 对象是 DSPy 中表示数据的核心类型。在DSPy中，你会频繁使用`Example`对象来表示数据集中的单个样本。每个 `Example` 对象代表一个数据样本，可以包含输入、中间标签和最终标签等信息。
   * `Example` 对象类似于 Python 字典，但它提供了一些额外的实用工具，例如 `with_inputs()` 方法，可以方便地标记哪些字段是输入字段。
   *  `Example` 对象用于表示训练集和测试集中的数据样本。DSPy 模块会返回 `Prediction` 类型的对象，它是 `Example` 的一个子类。


In [1]:
import dspy
#创建了一个 `Example` 对象，包含 `question` 和 `answer` 两个字段。
qa_pair = dspy.Example(question="这是一个问题吗？", answer="这是一个答案。")

print(qa_pair)            # 将会输出Example对象的详细信息
print(qa_pair.question)   # 访问字段值
print(qa_pair.answer)     # 访问字段值

Example({'question': '这是一个问题吗？', 'answer': '这是一个答案。'}) (input_keys=None)
这是一个问题吗？
这是一个答案。


`Example`对象的字段可以是任意键值对，值通常为字符串。定义时，可以使用类似函数调用的方式：

In [2]:
obj = dspy.Example(field1="值1", field2="值2", field3="值3")

你也可以用`Example`列表表示一个训练集：

In [4]:
trainset = [
    dspy.Example(report="长报告1", summary="简短摘要1"),
    dspy.Example(report="长报告2", summary="简短摘要2"),
    # ...
]

### 2. 指定输入键 (Specifying Input Keys)

*    在机器学习中，我们通常会区分「输入特征」和「标签」。在 DSPy 中，明确指定输入键可以帮助框架更好地理解数据的结构，并进行更有效的优化。
* `with_inputs()` 方法用于标记 `Example` 对象中的哪些字段是输入字段。其余的字段将被视为元数据或标签。


In [2]:
# 单一输入
print(qa_pair.with_inputs("question"))

# 多个输入；请小心标记您的标签为输入，除非您确实需要这样做。
print(qa_pair.with_inputs("question", "answer"))

Example({'question': '这是一个问题吗？', 'answer': '这是一个答案。'}) (input_keys={'question'})
Example({'question': '这是一个问题吗？', 'answer': '这是一个答案。'}) (input_keys={'answer', 'question'})



### 3. 访问和排除特定键

提取输入键或非输入键字段，从而便于后续处理。

*   `inputs()` 方法返回一个新的只包含输入键的 `Example` 对象。
*   `labels()` 方法返回一个新的只包含非输入键的 `Example` 对象。


In [3]:
article_summary = dspy.Example(article= "这是一篇文章。", summary= "这是一个摘要。").with_inputs("article")

input_key_only = article_summary.inputs()
non_input_key_only = article_summary.labels()

print("仅包含输入字段的 Example 对象:", input_key_only)
print("仅包含非输入字段的 Example 对象:", non_input_key_only)

仅包含输入字段的 Example 对象: Example({'article': '这是一篇文章。'}) (input_keys={'article'})
仅包含非输入字段的 Example 对象: Example({'summary': '这是一个摘要。'}) (input_keys=None)


In [7]:
# 从本地的 faq.md 文件中读取 FAQ
import re

with open("faq.md", "r", encoding="utf-8") as f:
    markdown_content = f.read()

def parse_questions(markdown_content):
    # 使用正则表达式提取所有 "#### Q: XXX" 格式的问题
    question_pattern = r'#### Q: (.+?)\n'
    questions = re.findall(question_pattern, markdown_content, re.DOTALL)
    return questions

questions = parse_questions(markdown_content)

# 展示前5个问题
questions[:5]

D:\Temp\ipykernel_36036\858339483.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='faq.md' mode='r' encoding='cp936'>
  with open("faq.md", "r", encoding="utf-8") as f:


['Why would I use Weaviate as my vector database?',
 'What is the difference between Weaviate and for example Elasticsearch?',
 'Do you offer Weaviate as a managed service?',
 'How should I configure the size of my instance?',
 'Do I need to know about Docker (Compose) to use Weaviate?']

In [8]:
# 将数据分为训练集、开发集和测试集
# 假设前20个用于训练，中间10个用于开发验证，剩下的用于测试
trainset = questions[:10]
devset = questions[10:15]
testset = questions[15:20]

# 使用dspy.Example封装问题，并指定哪些字段作为输入
trainset = [dspy.Example(question=q).with_inputs("question") for q in trainset]
devset = [dspy.Example(question=q).with_inputs("question") for q in devset]
testset = [dspy.Example(question=q).with_inputs("question") for q in testset]

print("开发集第一个样例：", devset[0])

开发集第一个样例： Example({'question': 'Do Weaviate classes have namespaces?'}) (input_keys={'question'})


*   `input_key_only` 变量包含一个只包含 `article` 字段的 `Example` 对象，因为 `article` 被标记为输入字段。
*   `non_input_key_only` 变量包含一个只包含 `summary` 字段的 `Example` 对象，因为 `summary` 不是输入字段。




## 定义并使用Metric（度量指标）

在 DSPy 中，指标不仅仅是衡量程序性能的工具，更是指导程序优化的罗盘。一个精心设计的指标能够准确地反映程序的优劣，并为 DSPy 的优化算法提供明确的方向。指标的重要性在于，它将抽象的任务目标转化为了具体的、可量化的评估标准，使得 DSPy 能够通过机器学习的方式自动提升程序性能。

 1. 什么是指标？如何为我的任务定义指标？

指标Metric是一个函数，它接收数据样本 ( `example` ) 和程序输出 ( `pred` ) 作为输入，并输出一个分数来量化输出的质量。这个分数可以是布尔值、整数或浮点数，具体取决于任务和评估的需求。

*   **如何根据任务需求定义合适的指标？** 定义指标的关键在于深入理解任务的目标。我们需要思考：什么样的输出是好的？什么样的输出是不好的？然后将这些判断标准转化为一个可量化的函数。对于简单的任务，例如分类或简短问答，可以使用准确率、精确率、召回率或 F1 分数等标准指标。对于复杂的任务，例如生成长文本，我们需要考虑多个方面的因素，例如文本的相关性、流畅性、逻辑性、是否包含有害信息等等，并设计一个能够综合评估这些因素的指标。

*   **如何编写一个简单的指标函数？** 一个简单的指标函数只需要接收 `example` 和 `pred` 两个参数，并返回一个分数。`trace` 参数通常在评估时可以忽略，用于在编译阶段追踪程序的执行过程，记录每个预测器的输入和输出。这对于在优化过程中验证中间步骤非常有用。


例如这个指标函数评估了 `example.answer` 和 `pred.answer` 是否相同（忽略大小写）。它返回一个布尔值，表示答案是否匹配。

In [ ]:
def validate_answer(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

In [ ]:
from dspy.evaluate import SemanticF1

# 初始化一个评估指标
metric = SemanticF1()

# module执行的结果
pred = dspy.ChainOfThought(**example.inputs())

# 开始评估
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")


同时 DSPy 提供了一些内置的实用工具：

*   `dspy.evaluate.metrics.answer_exact_match` 用于评估预测的答案是否与标准答案完全匹配。
*   `dspy.evaluate.metrics.answer_passage_match` 用于评估预测的答案是否与标准答案中的某个段落匹配。


In [ ]:
import dsp

def answer_exact_match(example, pred, trace=None, frac=1.0):
    """
    判断预测答案 (pred.answer) 是否与标准答案 (example.answer) 精确匹配。

    Args:
        example: 包含标准答案的样本数据，通常包含字段 example.answer。
        pred: 包含预测答案的预测结果，通常包含字段 pred.answer。
        trace:  可选参数，用于跟踪执行过程 (未使用)。
        frac: 匹配的阈值，当 example.answer 为字符串时有效，控制部分匹配的程度。

    Returns:
        bool: 如果预测答案与标准答案精确匹配，则返回 True，否则返回 False。
    """
    assert(type(example.answer) is str or type(example.answer) is list)

    if type(example.answer) is str:
        # 如果标准答案是字符串，则将标准答案包装成列表传递给 dsp.answer_match 函数
        return dsp.answer_match(pred.answer, [example.answer], frac=frac)
    else: # type(example.answer) is list
        # 如果标准答案是列表，则直接将预测答案和标准答案列表传递给 dsp.answer_match 函数
        return dsp.answer_match(pred.answer, example.answer, frac=frac)

# answer_exact_match_str 是 dsp.answer_match 的别名，用于处理字符串类型的标准答案的精确匹配
answer_exact_match_str = dsp.answer_match

def answer_passage_match(example, pred, trace=None):
    """
    判断预测的上下文 (pred.context) 中是否包含标准答案 (example.answer)。

    Args:
        example: 包含标准答案的样本数据，通常包含字段 example.answer。
        pred: 包含预测上下文的预测结果，通常包含字段 pred.context。
        trace: 可选参数，用于跟踪执行过程 (未使用)。

    Returns:
        bool: 如果预测的上下文中包含标准答案，则返回 True，否则返回 False。
    """
    assert(type(example.answer) is str or type(example.answer) is list)

    if type(example.answer) is str:
        # 如果标准答案是字符串，则将标准答案包装成列表传递给 dsp.passage_match 函数
        return dsp.passage_match(pred.context, [example.answer])
    else: # type(example.answer) is list
        # 如果标准答案是列表，则直接将预测的上下文和标准答案列表传递给 dsp.passage_match 函数
        return dsp.passage_match(pred.context, example.answer)

def passage_match(passages: list[str], answers: list[str]) -> bool:
    """
    判断给定的 passages 列表中是否包含任意一个答案。

    Returns:
        bool: 如果任意一个 passage 包含答案列表中的任意一个答案，则返回 True，否则返回 False。
    """
    return any(passage_has_answers(psg, answers) for psg in passages)

def answer_match(prediction, answers, frac=1.0):
    """
    根据 frac 参数的值，判断预测答案是否与标准答案匹配。
    如果 frac >= 1.0，则使用精确匹配 (EM)；
    否则，使用 F1 分数，并根据 frac 阈值判断是否匹配。

    Returns:
        bool: 如果预测答案与标准答案匹配，则返回 True，否则返回 False。
    """
    # pred = example.prediction
    # answers = example.answers

    if frac >= 1.0:
        # 如果 frac 大于等于 1.0，则使用精确匹配 (EM)
        return EM(prediction, answers)

    # 否则，使用 F1 分数，并根据 frac 阈值判断是否匹配
    return F1(prediction, answers) >= frac

返回对错有什么用呢？我们来看下面的例子，只看整个框架就行，并不具体讲到评估

In [ ]:
def evaluateAnswer(fn, dev, metric=EM):
    data = []

    for example in tqdm.tqdm(dev):
        question = example.question
        prediction = fn(question)

        d = dict(example)

        pred = prediction.answer

        d['prediction'] = pred
        #比较预测答案 pred 和标准答案 example.answer
        # 返回一个布尔值，True 表示预测正确，False 表示预测错误。存储在 d['correct'] 里
        d['correct'] = metric(pred, example.answer) 
        data.append(d)

    df = pd.DataFrame(data)
# df['correct'].sum()统计了所有 True 的数量，也就是答对的题目数量。总的是计算了正确率，也就是答对题目数量占总题目数量的百分比。
    percentage = round(100.0 * df['correct'].sum() / len(dev), 1)
    print(f"Answered {df['correct'].sum()} / {len(dev)} ({percentage}%) correctly.")
    df['correct'] = df['correct'].apply(lambda x: '✔️' if x else '❌')

    pd.options.display.max_colwidth = None
    ipython_display(df.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}, {'selector': 'td', 'props': [('text-align', 'left')]}]))


In [ ]:
def answer_match(prediction, answers, frac=1.0):
    # pred = example.prediction
    # answers = example.answers

    if frac >= 1.0:
        return EM(prediction, answers)

    return F1(prediction, answers) >= frac

具体来讲，里面的EM和F1是库里面内置的函数，比如有：
*   `EM`：精确匹配，适用于要求答案完全正确的场景。
*   `F1`：综合考虑精确率和召回率（recall），适用于答案部分匹配也可以得分的场景。
*   `HotPotF1`：针对 HotpotQA 数据集的 F1 得分，适用于评估在该数据集上的问答性能。
*   `nF1`：考虑对话历史的新颖 F1 得分，适用于评估对话系统中生成答案的新颖性和相关性。

机器学习经典的逻辑回归算法或其他二分类算法，经常用混淆矩阵来评估效果，比如这里算出来的F1得分

In [ ]:
from dsp.utils.metrics import em_score, f1_score, hotpot_f1_score, novel_f1_score

def EM(prediction, answers_list):
    assert type(answers_list) == list

    return max(em_score(prediction, ans) for ans in answers_list)


def F1(prediction, answers_list):
    assert type(answers_list) == list

    return max(f1_score(prediction, ans) for ans in answers_list)


def HotPotF1(prediction, answers_list):
    assert type(answers_list) == list

    return max(hotpot_f1_score(prediction, ans) for ans in answers_list)


def nF1(history, prediction, answers_list, return_recall=False):
    assert type(answers_list) == list

    return max(novel_f1_score(history, prediction, ans, return_recall=return_recall) for ans in answers_list)


DSPy经常用于rag等大项目优化中，我们常常要对文字做评估。

F1 分数 通过统计预测文本和标准答案中共同出现的词的数量来衡量二者的相似度。它在自然语言处理中被广泛用于评估各种任务的性能，例如问答、文本分类等。

F1 分数可以更全面地评估预测结果和标准答案的相似度，比单纯的精确匹配更灵活。

`f1_score` 就像一个既看查准率又看查全率的评分员。想象一下，预测文本和标准答案都是一袋子词语：

*   **查准率 (precision)：** 预测文本这袋子里有多少词语是和标准答案那袋子一样的。
*   **查全率 (recall)：** 标准答案那袋子里有多少词语是在预测文本这袋子里出现的。
*   **F1 分数：** 综合考虑这两个指标，得出一个总体的相似度评分。

In [1]:
# 库里面自带的具体实现
def f1_score(prediction, ground_truth):
   #  首先，`normalize_text` 函数被调用，用于清洗和标准化输入的文本。这一步至关重要，因为它确保比较是在一个公平的基础上进行的。例如，它可能会将所有文本转换为小写，删除标点符号，或者处理特殊字符。标准化之后，`.split()` 方法将文本分割成单词列表，这里称为 "tokens"。
   # 为什么要进行文本规范化呢？想象一下，如果没有这一步，"Apple" 和 "apple" 会被视为不同的单词，这将导致计算结果不准确。规范化确保我们关注的是单词的本质内容，而不是它们的形式变化。
    prediction_tokens = normalize_text(prediction).split()
    ground_truth_tokens = normalize_text(ground_truth).split()
    
   #   `Counter` 是 Python 的 `collections` 模块中的一个类，用于计算每个 token 出现的次数。`&` 操作符用于找出 `prediction_tokens` 和 `ground_truth_tokens` 中共同出现的 tokens 及其最小出现次数。然后，`sum(common.values())` 计算共同 tokens 的总数。
   #  这里的 `&` 操作是怎么工作的呢？`Counter` 对象之间的 `&` 运算实际上是在进行交集操作。它会返回一个新的 `Counter` 对象，其中包含了两个 `Counter` 对象中都存在的元素，并且每个元素的计数是两个 `Counter` 对象中该元素计数的最小值。

    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

   # 如果预测文本和标准答案都为空或如果预测文本和标准答案没有共同的词，则 F1 分数为 0。
    if len(prediction_tokens) == len(ground_truth_tokens) == 0:
        # Unlike most tasks, QReCC and SQuAD-2.0 assign 1.0 in this edge case. We don't for uniformity.
        print_message(
            "\n#> F1 Metric: Rare edge case of len(prediction_tokens) == len(ground_truth_tokens) == 0.\n")

    if num_same == 0:
        return 0


# 如果没有共同的 tokens，F1 分数直接返回 0。否则，计算精确率（precision）和召回率（recall）。精确率是预测中正确的比例，召回率是真实值中被正确预测的比例。F1 分数是精确率和召回率的调和平均数，用于平衡两者。
#  精确率和召回率分别代表什么呢？精确率衡量的是模型预测的准确性，即“模型说相关的东西里，有多少是真的相关的”；而召回率衡量的是模型找到所有相关内容的能力，即“所有真正相关的东西里，模型找到了多少”。F1 分数试图同时考虑这两者，提供一个综合的性能指标。
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

上面是截取库里面的代码，下面我们真正应用打分评估：

In [5]:
import dspy
from dspy.evaluate import SemanticF1


llm = dspy.LM(model="gpt-4o-mini", max_tokens=256)
dspy.settings.configure(lm=llm)

# 定义一个简单的 ChainOfThought 模块 (问答任务)
class QA(dspy.Signature):
    """回答问题，答案简短。"""
    question = dspy.InputField()
    response = dspy.OutputField()

# 中文示例
example1 = dspy.Example(
    question="法国的首都是什么？",
    response="法国的首都是巴黎。",
).with_inputs("question")

example2 = dspy.Example(
    question="世界上最高的山峰是什么？",
    response="世界上最高的山峰是珠穆朗玛峰，它位于喜马拉雅山脉。",
).with_inputs("question")

example3 = dspy.Example(
    question="谁画了蒙娜丽莎？",
    response="蒙娜丽莎是达芬奇画的。",
).with_inputs("question")

# 初始化 SemanticF1 指标 
metric = SemanticF1()

# 使用 ChainOfThought 模块生成预测
cot = dspy.ChainOfThought(QA)
# cot = dspy.ChainOfThought('question -> response') 同义

# 针对每个示例进行评估
for example in [example1, example2, example3]:
    # cot(question="法国的首都是什么？")
    pred = cot(question=example.question)  # 生成预测
    score = metric(example, pred)  # 使用 SemanticF1 评估

    print(f"Question: \t {example.question}\n")
    print(f"Gold Response: \t {example.response}\n")
    print(f"Predicted Response: \t {pred.response}\n")
    print(f"Semantic F1 Score: {score:.2f}\n")

    print("-" * 20)  # 分隔符

Question: 	 法国的首都是什么？

Gold Response: 	 法国的首都是巴黎。

Predicted Response: 	 巴黎。

Semantic F1 Score: 0.67

--------------------
Question: 	 世界上最高的山峰是什么？

Gold Response: 	 世界上最高的山峰是珠穆朗玛峰，它位于喜马拉雅山脉。

Predicted Response: 	 世界上最高的山峰是珠穆朗玛峰。

Semantic F1 Score: 0.67

--------------------
Question: 	 谁画了蒙娜丽莎？

Gold Response: 	 蒙娜丽莎是达芬奇画的。

Predicted Response: 	 蒙娜丽莎是列奥纳多·达·芬奇画的。

Semantic F1 Score: 1.00

--------------------


理想的Metric通常无法一次性就设计完美。建议从简单指标入手，然后在查看实际数据和系统输出后不断改进。

在更复杂的任务中，你可能需要同时检查多个条件。比如下面这个Metric同时检查:
   * 一是预测的答案是否与标准答案相同（忽略大小写）；
   * 二是预测的答案是否出现在检索到的上下文中。


In [ ]:
def validate_context_and_answer(example, pred, trace=None):
    # 检查黄金标签和预测答案是否相同
    answer_match = example.answer.lower() == pred.answer.lower()

    # 检查预测答案是否来自检索到的上下文之一
    context_match = any((pred.answer.lower() in c) for c in pred.context)

    if trace is None: # 如果我们正在进行评估或优化
        return (answer_match + context_match) / 2.0
    else: # 如果我们正在进行自举，即自生成每个步骤的良好演示
        return answer_match and context_match

*   `trace` 参数用于区分评估/优化模式和自举模式。在评估/优化模式下，它返回两个方面的平均分数。在自举模式下，它返回两个方面是否都满足的布尔值。



### 进行评估（Evaluation）

*   ** 提问思考：** 如何使用上面定义的指标进行评估？

    *   可以使用一个简单的 Python 循环来遍历开发集，运行程序，并使用指标函数计算每个样本的分数。


In [ ]:
scores = []
for x in devset:
    pred = program(**x.inputs())
    score = metric(x, pred)
    scores.append(score)

遍历开发集中的每个样本 `x`，使用 `x` 的输入运行程序 `program`，得到预测结果 `pred`，然后使用指标函数 `metric` 计算 `pred` 相对于 `x` 的分数，并将分数添加到 `scores` 列表中。

DSPy 提供了 `Evaluate` 工具类来简化评估过程：

In [ ]:
from dspy.evaluate import Evaluate

# 设置评估器，可以在代码中重复使用。
evaluator = Evaluate(devset=你的开发集, num_threads=1, display_progress=True, display_table=5)

# 启动评估。
evaluator(你的程序, metric=你的指标)

*   这里的`display_progress=True`会显示评估过程进度，`display_table=5`则会展示示例输出的表格情况（例如显示前5个样本的输入、输出和得分）。

* ** 提问思考：** `Evaluate` 工具类提供了哪些功能？

    *   `Evaluate` 工具类提供了并行评估、显示进度条、显示样本输入/输出以及指标分数等功能。

使用 AI 反馈作为指标


In [4]:
import dspy
from dspy.evaluate import Evaluate, SemanticF1

llm = dspy.LM(model="gpt-4o-mini", max_tokens=256)
dspy.settings.configure(lm=llm)

# 定义一个简单的 ChainOfThought 模块 (问答任务)
class QA(dspy.Signature):
    """回答问题，答案简短。"""
    question = dspy.InputField()
    response = dspy.OutputField()

# 中文示例
example1 = dspy.Example(
    question="法国的首都是什么？",
    response="法国的首都是巴黎。",
).with_inputs("question")

example2 = dspy.Example(
    question="世界上最高的山峰是什么？",
    response="世界上最高的山峰是珠穆朗玛峰，它位于喜马拉雅山脉。",
).with_inputs("question")

example3 = dspy.Example(
    question="谁画了蒙娜丽莎？",
    response="蒙娜丽莎是达芬奇画的。",
).with_inputs("question")

# 定义一个评估函数（可选）
def evaluate_qa(qa_module, devset, num_threads=1, display_progress=True, display_table=0):
    evaluator = Evaluate(
        devset=devset,
        num_threads=num_threads,
        display_progress=display_progress,
        display_table=display_table,
    )
    return evaluator(qa_module)

# 初始化 SemanticF1 指标
metric = SemanticF1()

# 构造一些预测结果，模拟 module 的输出
cot = dspy.ChainOfThought(QA)

pred1 = cot(question=example1.question)
pred2 = cot(question=example2.question)
pred3 = cot(question=example3.question)

# 评估并打印结果
print("Example 1:")
score1 = metric(example1, pred1)
print(f"Question: \t {example1.question}\n")
print(f"Gold Response: \t {example1.response}\n")
print(f"Predicted Response: \t {pred1.response}\n")
print(f"Semantic F1 Score: {score1:.2f}\n")


print("\nExample 2:")
score2 = metric(example2, pred2)
print(f"Question: \t {example2.question}\n")
print(f"Gold Response: \t {example2.response}\n")
print(f"Predicted Response: \t {pred2.response}\n")
print(f"Semantic F1 Score: {score2:.2f}\n")


print("\nExample 3:")
score3 = metric(example3, pred3)
print(f"Question: \t {example3.question}\n")
print(f"Gold Response: \t {example3.response}\n")
print(f"Predicted Response: \t {pred3.response}\n")
print(f"Semantic F1 Score: {score3:.2f}\n")


# 评估整个数据集 (可选)
# evaluate_qa(ChainOfThought(QA), devset=[example1, example2, example3], metric=metric)

Example 1:
Question: 	 法国的首都是什么？

Gold Response: 	 法国的首都是巴黎。

Predicted Response: 	 巴黎。

Semantic F1 Score: 0.67


Example 2:
Question: 	 世界上最高的山峰是什么？

Gold Response: 	 世界上最高的山峰是珠穆朗玛峰，它位于喜马拉雅山脉。

Predicted Response: 	 世界上最高的山峰是珠穆朗玛峰。

Semantic F1 Score: 0.67


Example 3:
Question: 	 谁画了蒙娜丽莎？

Gold Response: 	 蒙娜丽莎是达芬奇画的。

Predicted Response: 	 蒙娜丽莎是列奥纳多·达·芬奇画的。

Semantic F1 Score: 1.00




 对于复杂的任务，例如生成长文本，很难用简单的规则来评估输出的质量。AI 反馈可以利用大模型的理解能力，从多个维度评估输出的质量。
    *   **如何使用 AI 反馈来评估生成的推文？**  我们可以定义一个评估签名的类，并使用 `dspy.Predict` 来生成评估结果。然后，我们可以根据评估结果计算最终的分数。


业务场景：定义了一个Metric来检查预测的推文（tweet）是否回答了给定问题、是否具有吸引力（engaging），并且保证长度不超过280字符。

In [ ]:
# 定义自动评估的签名。
class Assess(dspy.Signature):
    """根据指定的维度评估推文的质量。"""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer: bool = dspy.OutputField()

In [ ]:
def metric(gold, pred, trace=None):
    question, answer, tweet = gold.question, gold.answer, pred.output

    engaging = "评估的文本是否构成了一个独立且引人入胜的推文？"
    correct = f"文本应该用“{answer}”回答“{question}”。评估的文本是否包含这个答案？"
    
    correct = dspy.Predict(Assess)(assessed_text=tweet, assessment_question=correct)
    engaging = dspy.Predict(Assess)(assessed_text=tweet, assessment_question=engaging)

    correct, engaging = [m.assessment_answer for m in [correct, engaging]]
    score = (correct + engaging) if correct and (len(tweet) <= 280) else 0

    if trace is not None: return score >= 2
    return score / 2.0

*   ** 提问思考：** 这段代码如何使用 AI 反馈来评估推文？`trace` 参数在其中的作用是什么？

    *   这段代码首先定义了两个评估问题：`engaging` 和 `correct`。然后，它使用 `dspy.Predict` 和 `Assess` 签名来生成对推文的评估结果。最后，它根据评估结果和推文长度计算最终的分数。
    *   `trace` 参数用于区分编译阶段和评估阶段。在编译阶段，只有当 `score >= 2` 时才返回 `True`，以确保生成的示例质量较高。在评估阶段，返回 `score / 2.0` 作为最终的分数。


